In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, pipeline
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch
import os

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

In [3]:
device

'cuda'

In [8]:
model_name = "HuggingFaceTB/SmolLM2-135M-Instruct"
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=model_name)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

In [10]:
%%time
prompt = "Explain AGI ?"
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0 if device == "cuda" else -1)
print(pipe(prompt, max_new_tokens=200))


Device set to use cuda:0


[{'generated_text': 'Explain AGI ?\nI am trying to decide between a AI and a human in this situation.\nAI is highly beneficial but I think humans can be too critical.\nHumans can be too critical but AI has many more benefits as well.\nMany people are unhappy or in pain with humans.\nIn the future, humans would love to have AI but I think humans would love AI.\n\nWhat would be the best position for both AI and humans?'}]
CPU times: total: 8.17 s
Wall time: 8.52 s


In [11]:
ds = load_dataset("prithivMLmods/Deepthink-Reasoning")

README.md: 0.00B [00:00, ?B/s]

C:\Users\Lenovo\Projects\smollm-finetune\.v\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Lenovo\.cache\huggingface\hub\datasets--prithivMLmods--Deepthink-Reasoning. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling

0000.parquet:   0%|          | 0.00/401k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/251 [00:00<?, ? examples/s]

In [12]:
def tokenize_function(examples):
    prompts = [p.strip() for p in examples["prompt"]]
    responses = [r.strip() for r in examples["response"]]
    texts = [
        tokenizer.apply_chat_template(
            [{"role": "user", "content": p}, {"role": "assistant", "content": r}],
            tokenize=False
        )
        for p, r in zip(prompts, responses)
    ]
    return tokenizer(texts, truncation=True, padding="max_length", max_length=512)

In [13]:
ds = ds.map(tokenize_function, batched=True)

Map:   0%|          | 0/251 [00:00<?, ? examples/s]